# 探索的データ分析 (EDA) - 三井物産商品価格予測チャレンジ

このノートブックでは、コンペティションデータの詳細な探索と分析を行います。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. データの読み込み

In [ ]:
# データの読み込み
try:
    train = pd.read_csv('../data/raw/train.csv')
    test = pd.read_csv('../data/raw/test.csv')
    sample_submission = pd.read_csv('../data/raw/sample_submission.csv')
    print("✅ データを正常に読み込みました")
except FileNotFoundError:
    print("⚠️ データファイルが見つかりません。")
    print("Kaggle APIを使用してデータをダウンロードしてください：")
    print("!kaggle competitions download -c mitsui-commodity-prediction-challenge")
    raise

## 2. データの基本情報

In [ ]:
print("=" * 50)
print("データセットの形状")
print("=" * 50)
print(f"訓練データ: {train.shape}")
print(f"テストデータ: {test.shape}")
print(f"サンプル提出: {sample_submission.shape}")

print("\n" + "=" * 50)
print("データ型")
print("=" * 50)
print(train.dtypes.value_counts())

print("\n" + "=" * 50)
print("訓練データの最初の5行")
print("=" * 50)
train.head()

## 3. 欠損値の分析

In [ ]:
def analyze_missing_values(df, title):
    missing = df.isnull().sum()
    missing_pct = 100 * missing / len(df)
    missing_table = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    })
    missing_table = missing_table[missing_table['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
    
    if len(missing_table) > 0:
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        missing_table['Missing_Percentage'].plot(kind='barh')
        plt.title(f'{title} - 欠損値の割合')
        plt.xlabel('欠損値の割合 (%)')
        
        plt.subplot(1, 2, 2)
        sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
        plt.title(f'{title} - 欠損値パターン')
        plt.tight_layout()
        plt.show()
    else:
        print(f"✅ {title}に欠損値はありません")
    
    return missing_table

missing_train = analyze_missing_values(train, '訓練データ')
missing_test = analyze_missing_values(test, 'テストデータ')

## 4. 統計的要約

In [ ]:
print("訓練データの統計的要約")
print("=" * 80)
train.describe(include='all')

## 5. ターゲット変数の分析

In [ ]:
if 'target' in train.columns:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('ターゲット分布', 'ボックスプロット', 'Q-Qプロット', '時系列')
    )
    
    # ヒストグラム
    fig.add_trace(
        go.Histogram(x=train['target'], nbinsx=50, name='Target'),
        row=1, col=1
    )
    
    # ボックスプロット
    fig.add_trace(
        go.Box(y=train['target'], name='Target'),
        row=1, col=2
    )
    
    # Q-Qプロット用データ準備
    from scipy import stats
    theoretical_quantiles = stats.probplot(train['target'], dist="norm")[0][0]
    sample_quantiles = stats.probplot(train['target'], dist="norm")[0][1]
    
    fig.add_trace(
        go.Scatter(x=theoretical_quantiles, y=sample_quantiles, mode='markers', name='Q-Q'),
        row=2, col=1
    )
    
    # 時系列（もし日付カラムがある場合）
    if 'date' in train.columns:
        fig.add_trace(
            go.Scatter(x=train['date'], y=train['target'], mode='lines', name='Time Series'),
            row=2, col=2
        )
    
    fig.update_layout(height=800, showlegend=False, title_text="ターゲット変数の詳細分析")
    fig.show()
    
    # 統計情報
    print("ターゲット変数の統計:")
    print(f"平均値: {train['target'].mean():.4f}")
    print(f"中央値: {train['target'].median():.4f}")
    print(f"標準偏差: {train['target'].std():.4f}")
    print(f"歪度: {train['target'].skew():.4f}")
    print(f"尖度: {train['target'].kurtosis():.4f}")

## 6. 特徴量の相関分析

In [ ]:
# 数値型の特徴量のみ選択
numeric_features = train.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_features) > 1:
    # 相関行列の計算
    correlation_matrix = train[numeric_features].corr()
    
    # ヒートマップ
    plt.figure(figsize=(15, 12))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='coolwarm', vmin=-1, vmax=1, center=0,
                square=True, linewidths=0.5)
    plt.title('特徴量の相関行列', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # ターゲットとの相関が高い特徴量
    if 'target' in train.columns:
        target_corr = correlation_matrix['target'].sort_values(ascending=False)
        print("\nターゲットとの相関 (Top 10):")
        print(target_corr.head(11)[1:])  # targetを除く

## 7. カテゴリカル変数の分析

In [ ]:
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

if categorical_features:
    print(f"カテゴリカル特徴量: {categorical_features}")
    
    for col in categorical_features[:5]:  # 最初の5つのみ表示
        print(f"\n{col}の値の分布:")
        print(train[col].value_counts().head(10))
        
        # カテゴリ数が少ない場合はプロット
        if train[col].nunique() <= 20:
            plt.figure(figsize=(10, 5))
            train[col].value_counts().plot(kind='bar')
            plt.title(f'{col}の分布')
            plt.xlabel(col)
            plt.ylabel('カウント')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
else:
    print("カテゴリカル特徴量はありません")

## 8. 時系列分析（日付カラムがある場合）

In [ ]:
if 'date' in train.columns:
    # 日付型に変換
    train['date'] = pd.to_datetime(train['date'])
    test['date'] = pd.to_datetime(test['date']) if 'date' in test.columns else None
    
    # 時系列の範囲
    print(f"訓練データの期間: {train['date'].min()} から {train['date'].max()}")
    if 'date' in test.columns:
        print(f"テストデータの期間: {test['date'].min()} から {test['date'].max()}")
    
    # 時系列でのデータポイント数
    train_daily = train.groupby('date').size()
    
    plt.figure(figsize=(15, 5))
    plt.plot(train_daily.index, train_daily.values)
    plt.title('日次データポイント数')
    plt.xlabel('日付')
    plt.ylabel('データポイント数')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 曜日・月による分析
    train['dayofweek'] = train['date'].dt.dayofweek
    train['month'] = train['date'].dt.month
    train['year'] = train['date'].dt.year
    
    if 'target' in train.columns:
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # 曜日別
        train.groupby('dayofweek')['target'].mean().plot(kind='bar', ax=axes[0])
        axes[0].set_title('曜日別平均ターゲット')
        axes[0].set_xlabel('曜日 (0=月曜日)')
        
        # 月別
        train.groupby('month')['target'].mean().plot(kind='bar', ax=axes[1])
        axes[1].set_title('月別平均ターゲット')
        axes[1].set_xlabel('月')
        
        # 年別
        train.groupby('year')['target'].mean().plot(kind='bar', ax=axes[2])
        axes[2].set_title('年別平均ターゲット')
        axes[2].set_xlabel('年')
        
        plt.tight_layout()
        plt.show()

## 9. 外れ値の検出

In [ ]:
def detect_outliers(df, features, n_std=3):
    outliers_dict = {}
    
    for col in features:
        if df[col].dtype in [np.int64, np.float64]:
            mean = df[col].mean()
            std = df[col].std()
            
            # Z-scoreを使用した外れ値検出
            z_scores = np.abs((df[col] - mean) / std)
            outliers = df[z_scores > n_std]
            
            if len(outliers) > 0:
                outliers_dict[col] = len(outliers)
    
    return outliers_dict

# 外れ値の検出
outliers = detect_outliers(train, numeric_features)

if outliers:
    outliers_df = pd.DataFrame(list(outliers.items()), columns=['Feature', 'Outlier_Count'])
    outliers_df['Outlier_Percentage'] = 100 * outliers_df['Outlier_Count'] / len(train)
    outliers_df = outliers_df.sort_values('Outlier_Percentage', ascending=False)
    
    print("外れ値の検出結果 (3σ基準):")
    print(outliers_df)
    
    # 可視化
    if len(outliers_df) > 0:
        plt.figure(figsize=(12, 6))
        plt.bar(range(len(outliers_df)), outliers_df['Outlier_Percentage'])
        plt.xticks(range(len(outliers_df)), outliers_df['Feature'], rotation=45)
        plt.title('特徴量別外れ値の割合')
        plt.ylabel('外れ値の割合 (%)')
        plt.tight_layout()
        plt.show()
else:
    print("外れ値は検出されませんでした（3σ基準）")

## 10. まとめと次のステップ

In [ ]:
print("=" * 80)
print("EDA サマリー")
print("=" * 80)

print(f"\n📊 データセット情報:")
print(f"  - 訓練サンプル数: {len(train):,}")
print(f"  - テストサンプル数: {len(test):,}")
print(f"  - 特徴量数: {train.shape[1]}")
print(f"  - 数値型特徴量: {len(numeric_features)}")
print(f"  - カテゴリカル特徴量: {len(categorical_features)}")

if len(missing_train) > 0:
    print(f"\n⚠️ 欠損値:")
    print(f"  - 欠損値を含む特徴量数: {len(missing_train)}")
    print(f"  - 最大欠損率: {missing_train['Missing_Percentage'].max():.2f}%")

if 'target' in train.columns:
    print(f"\n🎯 ターゲット変数:")
    print(f"  - 平均: {train['target'].mean():.4f}")
    print(f"  - 標準偏差: {train['target'].std():.4f}")
    print(f"  - 歪度: {train['target'].skew():.4f}")

print(f"\n📝 推奨される次のステップ:")
print("  1. 特徴量エンジニアリング")
print("  2. 欠損値の処理戦略の決定")
print("  3. 外れ値の処理")
print("  4. ベースラインモデルの構築")
print("  5. クロスバリデーション戦略の設計")